pascalVOCのフォーマットを読み取ってVoTTに表示、編集します。

In [ ]:
%cd C:\Users\spc134\Desktop\convert_pascalvoc_to_vott-master

C:\Users\spc134\Desktop\convert_pascalvoc_to_vott-master


In [ ]:
import os
import argparse
from os import scandir
import xml.etree.ElementTree as ET
from uuid import uuid1
import json
from random import randint


class Converter:
    # TAGS_LIST = ["0", "1", "2", "3"]
    TAGS_LIST = ["hinageshi", "noharagarashi", "fieldPancy", "nezumimugi"]

    def __init__(self, path_to_dataset_annotations='', output_dir=''):
        self.annotations = path_to_dataset_annotations
        self.output_dir = output_dir

    def process(self):
        with open('sample_vott.json') as sv:
            vott_data = json.load(sv)
            vott_data['assets'] = {}
            vott_data['tags'] = []
            for tag in Converter.TAGS_LIST:
                vott_data["tags"].append(
                    {
                        "name": tag,
                        "color": self.htmlcolor(randint(40, 200), randint(40, 200), randint(40, 200))
                    }
                )
            for idx, entry in enumerate(scandir(self.annotations)):
                if entry.name.endswith(".xml"):
                    print(entry.name)
                    print(entry.path)
                    asset_data = self.read_data_from_xml(entry.path)
                    if idx == 0:
                        vott_data['lastVisitedAssetId'] = asset_data['asset']['id']
                    vott_data['assets'][asset_data['asset']['id']] = asset_data['asset']
                    filename = f"{asset_data['asset']['id']}-asset.json"
                    print(filename)
                    with open(os.path.join(self.output_dir, filename), "w") as ov:
                        json.dump(asset_data, ov, indent=4, sort_keys=True)
            with open(os.path.join(self.output_dir, "output_vott.vott"), "w") as ov:
                json.dump(vott_data, ov, indent=4, sort_keys=True)

    def read_data_from_xml(self, path_to_xml):
        TAGS_list = ["hinageshi", "noharagarashi", "fieldPancy", "nezumimugi"]
        results = {}
        print('path to xml', path_to_xml)
        tree = ET.parse(path_to_xml)
        print(tree.getroot().find('path').text)
        root = tree.getroot()
        xml_attr = {}
        xml_attr['path'] = root.find('path').text
        print('find(path)', root.find('path').text)
        xml_attr['size'] = root.find('size')
        results['asset'] = {}
        results['asset']['format'] = xml_attr['path'].split('.')[-1]
        results['asset']['id'] = str(uuid1()).replace("-", "")
        results['asset']['name'] = xml_attr['path'].split(os.sep)[-1]
        results['asset']['path'] = "file:" + xml_attr['path'].replace("Annotations", "Images").replace("\\", "/")
        print('asset path', xml_attr['path'])
        results['asset']['size'] = {
            "width": int(xml_attr['size'].find('width').text),
            "height": int(xml_attr['size'].find('height').text)
        }
        results['asset']['state'] = 2
        results['asset']['type'] = 1
        results['regions'] = []
        for object in root.findall('object'):
            temp_region = {}
            temp_region['id'] = str(uuid1()).replace("-", "")
            temp_region['type'] = "RECTANGLE"
            temp_region['tags'] = []
            # temp_region['tags'] = list(object.find('name').text)
            temp_region['tags'].append(object.find('name').text)
            print(temp_region['tags'])
            obj_bnd_box = object.find('bndbox')
            xmin = int(obj_bnd_box.find('xmin').text)
            ymin = int(obj_bnd_box.find('ymin').text)
            xmax = int(obj_bnd_box.find('xmax').text)
            ymax = int(obj_bnd_box.find('ymax').text)

            temp_region["boundingBox"] = {
                "height": ymax - ymin,
                "width": xmax - xmin,
                "left": xmin,
                "top": ymin
            }
            temp_region['points'] = [
                {
                    "x": xmin,
                    "y": ymin
                },
                {
                    "x": xmax,
                    "y": ymin
                },
                {
                    "x": xmax,
                    "y": ymax
                },
                {
                    "x": xmin,
                    "y": ymax
                }
            ]

            results['regions'].append(temp_region)
        results['version'] = "2.1.0"
        return results

    def htmlcolor(self, r, g, b):
        def _chkarg(a):
            if isinstance(a, int):  # clamp to range 0--255
                if a < 0:
                    a = 0
                elif a > 255:
                    a = 255
            elif isinstance(a, float):  # clamp to range 0.0--1.0 and convert to integer 0--255
                if a < 0.0:
                    a = 0
                elif a > 1.0:
                    a = 255
                else:
                    a = int(round(a * 255))
            else:
                raise ValueError('Arguments must be integers or floats.')
            return a

        r = _chkarg(r)
        g = _chkarg(g)
        b = _chkarg(b)
        return '#{:02x}{:02x}{:02x}'.format(r, g, b)


parser = argparse.ArgumentParser()
parser.add_argument("--anno_path", type=str)
parser.add_argument("--out_dir", type=str)
args = parser.parse_args()
print('anno_path', args.anno_path)
converter = Converter(path_to_dataset_annotations=args.anno_path,
                      output_dir=args.out_dir)
converter.process()


In [ ]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is CE97-FE84

 Directory of C:\Users\spc134\Desktop\convert_pascalvoc_to_vott-master

11/06/2020  03:06 PM    <DIR>          .
11/06/2020  03:06 PM    <DIR>          ..
11/04/2020  01:45 PM             1,949 .gitignore
11/04/2020  01:45 PM    <DIR>          .idea
11/04/2020  01:45 PM    <DIR>          annotations_
11/06/2020  02:41 PM             5,405 main.py
11/04/2020  01:45 PM             1,957 readme.md
11/04/2020  01:45 PM    <DIR>          readme_images
11/06/2020  03:05 PM             2,214 sample_vott.json
11/04/2020  01:45 PM             3,063 sample_vott.json_
               5 File(s)         14,588 bytes
               5 Dir(s)  181,014,401,024 bytes free


参考。以下コード

In [ ]:
!python main.py --out_dir C:\Users\spc134\Desktop\makecoco\PascalVOC\Images --anno_path C:\Users\spc134\Desktop\makecoco\PascalVOC\Images

anno_path C:\Users\spc134\Desktop\makecoco\PascalVOC\Images
200423_7_1_03.xml
C:\Users\spc134\Desktop\makecoco\PascalVOC\Images\200423_7_1_03.xml
path to xml C:\Users\spc134\Desktop\makecoco\PascalVOC\Images\200423_7_1_03.xml
C:\Users\spc134\Desktop\makecoco\PascalVOC\Annotations\200423_7_1_03.png
find(path) C:\Users\spc134\Desktop\makecoco\PascalVOC\Annotations\200423_7_1_03.png
asset path C:\Users\spc134\Desktop\makecoco\PascalVOC\Annotations\200423_7_1_03.png
['hinageshi']
['hinageshi']
['hinageshi']
['hinageshi']
['hinageshi']
['hinageshi']
439c312e1ff611eb8796b052169989b0-asset.json
200423_7_1_04.xml
C:\Users\spc134\Desktop\makecoco\PascalVOC\Images\200423_7_1_04.xml
path to xml C:\Users\spc134\Desktop\makecoco\PascalVOC\Images\200423_7_1_04.xml
C:\Users\spc134\Desktop\makecoco\PascalVOC\Annotations\200423_7_1_04.png
find(path) C:\Users\spc134\Desktop\makecoco\PascalVOC\Annotations\200423_7_1_04.png
asset path C:\Users\spc134\Desktop\makecoco\PascalVOC\Annotations\200423_7_1_04.pn

In [ ]:
!findstr '200423_7_2_28' C:\Users\spc134\Desktop\makecoco\PascalVOC\Images